In [ ]:
pip install -U sentence-transformers

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
from sentence_transformers import SentenceTransformer
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
import time

import glob
import os
import nltk.data
import nltk
nltk.download('punkt')
import numpy as np
import tqdm

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

import pickle

import torch
import torch.nn as nn

In [2]:
#   reload pickled data from file
with open('train_inputs.pkl', 'rb') as f:
    train_inputs = pickle.load(f)

with open('train_labels.pkl', 'rb') as f:
    train_labels = pickle.load(f)               

In [16]:
#   reload pickled data from file
with open('test_inputs.pkl', 'rb') as f:
    test_inputs = pickle.load(f)

with open('test_labels.pkl', 'rb') as f:
    test_labels = pickle.load(f)               

In [ ]:
X = torch.tensor(train_inputs, dtype=torch.float32)
y = torch.tensor(train_labels, dtype=torch.float32)
y = y.unsqueeze(1)

In [15]:
class Deep(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(384*3, 18)
        self.act1 = nn.ReLU()
        self.layer2 = nn.Linear(18, 12)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(12, 6)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(6, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.sigmoid(self.output(x))
        return x

In [19]:
import copy
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm

def model_train(model, X_train, y_train, X_val, y_val):
    # loss function and optimizer
    loss_fn = nn.BCELoss()  # binary cross entropy
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    n_epochs = 1  # number of epochs to run
    batch_size = 10  # size of each batch
    batch_start = torch.arange(0, len(X_train), batch_size)

    # Hold the best model
    best_acc = - np.inf   # init to negative infinity
    best_weights = None

    for epoch in range(n_epochs):
        model.train()
        with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
            bar.set_description(f"Epoch {epoch}")
            for start in bar:
                # take a batch
                X_batch = X_train[start:start+batch_size]
                y_batch = y_train[start:start+batch_size]
                # forward pass
                y_pred = model(X_batch)
                loss = loss_fn(y_pred, y_batch)
                # backward pass
                optimizer.zero_grad()
                loss.backward()
                # update weights
                optimizer.step()
                # print progress
                acc = (y_pred.round() == y_batch).float().mean()
                bar.set_postfix(
                    loss=float(loss),
                    acc=float(acc)
                )
        # evaluate accuracy at end of each epoch
        model.eval()
        y_pred = model(X_val)
        acc = (y_pred.round() == y_val).float().mean()
        acc = float(acc)
        if acc > best_acc:
            best_acc = acc
            best_weights = copy.deepcopy(model.state_dict())
        print(acc)
    # restore model and return best accuracy
    model.load_state_dict(best_weights)
    return best_acc

In [ ]:
X_train = torch.tensor(train_inputs, dtype=torch.float32)
y_train = torch.tensor(train_labels, dtype=torch.float32)
y_train= y_train.unsqueeze(1)
X_train, _, y_train, _ = train_test_split(X_train, y_train, train_size=0.999, shuffle=True)
X_test = torch.tensor(test_inputs, dtype=torch.float32)
y_test = torch.tensor(test_labels, dtype=torch.float32)
y_test = y_test.unsqueeze(1)
model = Deep()
acc = model_train(model, X_train, y_train, X_test, y_test)
print(f"Final model accuracy: {acc*100:.2f}%")